In [ ]:
import os
from os.path import join
import numpy as np
import pandas as pd
import cv2

from src import config
from src.utils import RLenc

PREDICTION_DIR = '/workdir/data/predictions/train-folds-005'
FOLDS = [0, 1, 2, 3, 4]


segm_thresh = 0.5
prob_thresh = 0.5

In [ ]:
def get_mean_probs_df(pred_dir):
    probs_df_lst = []
    for i in range(len(FOLDS)):
        fold_dir = os.path.join(pred_dir, f'fold_{FOLDS[i]}')
        probs_df = pd.read_csv(join(fold_dir, 'test', 'probs.csv'), index_col='id')
        probs_df_lst.append(probs_df)

    mean_probs_df = probs_df_lst[0].copy()
    for probs_df in probs_df_lst[1:]:
        mean_probs_df.prob += probs_df.prob
    mean_probs_df.prob /= len(probs_df_lst)

    return mean_probs_df

In [ ]:
mean_probs_df = get_mean_probs_df(PREDICTION_DIR)

In [ ]:
sample_submition = pd.read_csv(config.SAMPLE_SUBM_PATH)

for i, row in sample_submition.iterrows():
    pred_name = row.id+'.npy'
    pred_lst = []
    for i in range(len(FOLDS)):
        fold_dir = os.path.join(PREDICTION_DIR, f'fold_{FOLDS[i]}')
        pred_path = join(fold_dir, 'test', pred_name)
        pred = np.load(pred_path)
        pred_lst.append(pred)
        
    mean_pred = np.mean(pred_lst, axis=0)
    prob = mean_probs_df.loc[row.id].prob
    
    pred = mean_pred > segm_thresh
    prob = int(prob > prob_thresh)
    pred = (pred * prob).astype(np.uint8)

    rle_mask = RLenc(pred)
    row.rle_mask = rle_mask

sample_submition.to_csv('/workdir/data/submissions/train-folds-005-0.45-mean.csv', index=False) 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.hist(mean_probs_df.prob.values, bins=20)

In [ ]:
sample_submition = pd.read_csv(config.SAMPLE_SUBM_PATH)
input_dir = '/workdir/data/predictions/mean-005-0.40_draw/'
for i, row in sample_submition.iterrows():
    pred_name = row.id+'.png'
    pred = cv2.imread(join(input_dir, pred_name), cv2.IMREAD_GRAYSCALE)
    pred = pred > 0
    rle_mask = RLenc(pred.astype(np.uint8))
    row.rle_mask = rle_mask

sample_submition.to_csv('/workdir/data/submissions/mean-005-0.40_draw.csv', index=False) 

In [ ]:
sample_submition.to_csv('/workdir/data/submissions/mean-003-0.40_draw.csv', index=False) 